In [1]:
from pathlib import Path as p
from glob import glob
import json
import shutil as sh
import warnings
from tqdm import tqdm
from PIL import Image as I
from matplotlib import pyplot as plt
import imagesize

warnings.filterwarnings("ignore")

In [2]:
dataset_root_dir = "custom_dataset"
sh.rmtree(dataset_root_dir)
train_images_dir = "images/train"
val_images_dir = "images/val"
test_images_dir = "images/test"

p.mkdir(p(dataset_root_dir), exist_ok=True, parents=True)

for path in [train_images_dir, val_images_dir, test_images_dir]:
    p.mkdir(p(dataset_root_dir) / path, exist_ok=True, parents=True)

yaml_content = f"""
path: ../{dataset_root_dir} 
train: {train_images_dir} 
val: {val_images_dir}
test:  {test_images_dir}
names:
  0: dummy
  1: plate
"""

with open("custom_dataset.yaml", "w") as f:
    f.write(yaml_content)

In [3]:
source_dataset = "nomeroff_datasets/autoriaNumberplateDataset-2022-08-01/"
for subset in ["train", "val"]:
    all_images_paths = [
        path
        for path in glob(
            f"nomeroff_datasets/autoriaNumberplateDataset-2022-08-01/{subset}/*"
        )
        if ".json" not in path and ".sh" not in path
    ]
    annotations_path = [
        path
        for path in glob(
            f"nomeroff_datasets/autoriaNumberplateDataset-2022-08-01/{subset}/*"
        )
        if ".json" in path
    ][0]
    with open(annotations_path, "r") as f:
        annotation = json.loads(f.read())
    ann_dir = dataset_root_dir + "/" + f"labels/{subset}" + "/"

    p.mkdir(p(ann_dir), parents=True, exist_ok=True)

    nr = [
        (fn["filename"], fn["regions"])
        for fn in list(annotation["_via_img_metadata"].values())
    ]
    for filename, regions in tqdm(nr):

        s = source_dataset + subset + "/" + filename
        d = dataset_root_dir + "/" + f"images/{subset}" + "/" + filename
        sh.copy(s, d)
        image_width, image_height = imagesize.get(s)
        ann_content = ""
        for region in regions:
            shape_attributes = region["shape_attributes"]
            if shape_attributes["name"] == "polygon":
                all_points_x = region["shape_attributes"]["all_points_x"]
                all_points_y = region["shape_attributes"]["all_points_y"]
                width = (max(all_points_x) - min(all_points_x)) / image_width
                height = (max(all_points_y) - min(all_points_y)) / image_height

                x_center = min(all_points_x) / image_width + width / 2
                y_center = min(all_points_y) / image_height + height / 2

                ann_content += "1 {:.6f} {:.6f} {:.6f} {:.6f}\n".format(
                    x_center, y_center, width, height
                )

        basename = ".".join(filename.split(".")[:-1])
        txt_save_path = ann_dir + basename + ".txt"

        with open(txt_save_path, "w") as f:
            f.write(ann_content)

100%|██████████| 2261/2261 [01:05<00:00, 34.49it/s]


In [4]:
! python yolov5/train.py --img 640 --batch 8 --epochs 30 --data custom_dataset.yaml --weights yolov5s.pt --project . --name yolov5s_train

train: weights=yolov5s.pt, cfg=, data=custom_dataset.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=30, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=., name=yolov5s_train, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-159-gd669a74 Python-3.8.13 torch-1.12.1 CUDA:0 (NVIDIA GeForce RTX 2070 SUPER, 8192MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=